In [1]:
import sys
sys.path.append('C:/Users/logan/Desktop/PhDResearch')

import numpy as np 
import sympy 

# other files I created 
import header
import linearize

#### System Statistics

In [ ]:
#variances
varV = .1   # msmt. noise variance 
vphi = 1  # phase process noise in rads/s
vfd  = 2  # doppler frequency process noise in Hz
vfdd = 1  # doppler rate process noise in Hz 
vbias = 1 # variance of non-time-correlated portion of IMU biasing on estimates  

a, phi, b, phi_0, fd, fddot= sympy.symbols('a phi b phi_0 fd fddot')
vars = list([phi, fd, fddot, b]) # state variables 
Ts = .1  # sample time 
Tc = .02 # coherent integration time 
k = np.linspace(0, 10, num = int(10/Ts))

#### Note the doppler changes from the IMU bias
Doppler frequency equation is: 

> $f_d = \frac{v +/- v_o}{v +/- v_s}$ 

Where $f_d$ is the doppler frequency, $v$ is the speed of sound, $v_o$ is the speed of the observer and $v_s$ is the speed of the source

*the IMU bias on the velocity will adjust this measurement*

> $f_d = \frac{v +/- (v_o+b)}{v +/- v_s}$

As a simple additive process to measurements of the observer speed. 
The bias process is modeled as follows: 

>$\dot{b} = \frac{1}{T_c}b + \omega$

#### The Dynamics and Measurement Model

The phase will be estimated by the following process: 

> $\phi_{k} = \phi_{0} + 2\pi(f_{d,k}kTs + \frac{1}{2} \dot{f}_{d,k} k^2 Ts^2)$

using 

> $f_d = \frac{v + (v_o+b)}{v + v_s}$
>> *where* $f_{d,k} = f_{d,k-1} + Ts*\dot{f}_{d,k-1}$

Noting that: 

> $\dot{f}_d = \frac{-\dot{v}_s(v + (v_o+b))}{(v + v_s)^2} + \frac{(\dot{v}_0 + \dot{b})}{v + v_s}= \frac{-\dot{v}_s(v + (v_o+b)) + (\dot{v}_0 + \dot{b})(v + v_s)}{(v + v_s)^2}$
>> noting $\dot{b} = \frac{1}{T_c}b + \omega$

In [4]:
# measurements 
# linearize the observation matrix 
H = sympy.Matrix([[a*sympy.sin(phi)],[a*sympy.cos(phi)]])  # observation matrix 
dH, varsa = linearize.linearize(H, vars)
D = sympy.diag(varV*np.ones((1,np.transpose(H).shape[1])))

# dynamics 
A = sympy.Matrix([[1, Ts, (Ts**2)/2],[0, 1, Ts],[0, 0, 1]]) # state transition matrix
nx = np.transpose(A).shape[1]         # number of states must match columns of A 
B = sympy.Matrix(np.zeros((A.shape))) # control input matrix (sparse)
G = sympy.eye(nx)                     # process noise input matrix
ng = np.transpose(G).shape[1]         # number of noise variables must match the colummns of G 

# store all system information in a dataclass 
sys = header.SymSystem(A, B, G, dH, D) # NOTE that dH is linearized, and the states there are defined by their deviation from some nominal value 